# Ansbile playbook

## 1. Simple playbook

### 1.1. Run playbook with current user

In [ ]:
ansible-playbook ping.yml

### 1.2. Run playbook with become user

In [ ]:
ansible-playbook ping_root.yml --vault-id vault-id -e "@password.yml"

### 1.3. Show ansible variables

In [ ]:
ansible-playbook vars.yml